In [5]:
import numpy as np
import pandas as pd

num_servers = 100000
cpu_load_range = (0.0, 100.0)
cpu_cores_range = (1, 8)
memory_ram_range = (0.0, 100.0)
memory_swap_range = (0.0, 100.0)
disk_usage_range = (0.0, 100.0)

cpu_load = np.random.uniform(cpu_load_range[0], cpu_load_range[1], size=num_servers)
cpu_cores = np.random.randint(cpu_cores_range[0], cpu_cores_range[1] + 1, size=num_servers)
memory_ram = np.random.uniform(memory_ram_range[0], memory_ram_range[1], size=num_servers)
memory_swap = np.random.uniform(memory_swap_range[0], memory_swap_range[1], size=num_servers)
disk_usage = np.random.uniform(disk_usage_range[0], disk_usage_range[1], size=num_servers)

df = pd.DataFrame({'CPU Load (%)': cpu_load.round(2),
                   'CPU Cores': cpu_cores,
                   'Memory RAM Usage (%)': memory_ram.round(2),
                   'Memory Swap Usage (%)': memory_swap.round(2),
                   'Disk Usage (%)': disk_usage.round(2)})

df['Downgrade'] = ((df['CPU Load (%)'] > 70.0) | (df['Memory Swap Usage (%)'] > 50.0) | (df['CPU Cores'] > 16) | ((df['Memory RAM Usage (%)'] + df['Memory Swap Usage (%)']) > 70.0) | (df['Disk Usage (%)'] > 80.0))

In [30]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score, 
    f1_score, 
    precision_score, 
    recall_score
)
import mlflow


class NeuralNetwork:
    def __init__(self, num_classes, hidden_layers=(256, 128), learning_rate=0.001):
        self.num_classes = num_classes
        self.hidden_layers = hidden_layers
        self.learning_rate = learning_rate
        self.model = self._build_model()
        self.tracking = mlflow.set_tracking_uri("http://localhost:5000")
        self.experiment = mlflow.set_experiment("NeuralNetwork")

    def _build_model(self):
        model = MLPClassifier(hidden_layer_sizes=self.hidden_layers,
                              activation='relu', solver='adam',
                              learning_rate_init=self.learning_rate, 
                              max_iter=200)
        return model

    def train(self, X_train, y_train, X_val, y_val):
        with mlflow.start_run():
            self.model.fit(X_train, y_train)
            
            y_pred = self.model.predict(X_val)
            accuracy = accuracy_score(y_val, y_pred)
            f1 = f1_score(y_val, y_pred)
            precision = precision_score(y_val, y_pred)
            recall = recall_score(y_val, y_pred)
            
            mlflow.log_metric("accuracy", accuracy)
            mlflow.log_metric("f1_score", f1)
            mlflow.log_metric("precision", precision)
            mlflow.log_metric("recall", recall)
            
            mlflow.sklearn.log_model(self.model, "model")
            
            return accuracy, f1, precision, recall

    def predict(self, X_new):
        return self.model.predict(X_new)

In [ ]:
df['downgrade'].value_counts()

In [7]:
df['downgrade'] = df['Downgrade'].astype(int)

In [8]:
df.head()

,CPU Load (%),CPU Cores,Memory RAM Usage (%),Memory Swap Usage (%),Disk Usage (%),Downgrade,downgrade
0,86.81,2,86.34,79.05,54.58,True,1
1,82.48,4,99.86,31.85,84.87,True,1
2,70.12,4,94.80,11.67,93.15,True,1
3,19.17,4,20.22,55.62,14.25,True,1
4,41.22,3,88.39,3.89,82.50,True,1


In [9]:
X = df[['CPU Load (%)', 'CPU Cores', 'Memory RAM Usage (%)','Memory Swap Usage (%)','Disk Usage (%)']]
y = df['downgrade']

In [32]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [33]:
nn = NeuralNetwork(num_classes=6)

In [34]:
accuracy, f1, precision, recall = nn.train(X_train, y_train, X_test, y_test)

In [35]:
print(f"Accuracy: {accuracy}, F1-score: {f1}, Precision: {precision}, Recall: {recall}")

Accuracy: 0.9914, F1-score: 0.995061444814517, Precision: 0.9994808790448174, Recall: 0.9906809216168315


In [36]:
y_pred = nn.predict(X_test)
print(f"Predictions: {y_pred}")

Predictions: [0 1 1 ... 1 1 1]


In [59]:
import numpy as np
import mlflow
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.neural_network import MLPClassifier


class NeuralNetwork:
    def __init__(self, hidden_layer_sizes=(100,), max_iter=200, random_state=42):
        self.hidden_layer_sizes = hidden_layer_sizes
        self.max_iter = max_iter
        self.random_state = random_state
        self.model = None
        self.tracking = mlflow.set_tracking_uri("http://localhost:5000")
        self.experiment = mlflow.set_experiment("NeuralNetwork")

    def _build_model(self, input_shape, output_shape):
        model = MLPClassifier(hidden_layer_sizes=self.hidden_layer_sizes, max_iter=self.max_iter, 
                              random_state=self.random_state)
        return model

    def fit(self, X_train, y_train, X_val, y_val):
        with mlflow.start_run():

            # registra os hiperparâmetros
            mlflow.log_param("hidden_layer_sizes", self.hidden_layer_sizes)
            mlflow.log_param("max_iter", self.max_iter)
            mlflow.log_param("random_state", self.random_state)

            # constrói o modelo
            input_shape = X_train.shape[1:]
            output_shape = 1
            self.model = self._build_model(input_shape, output_shape)

            # treina o modelo
            self.model.fit(X_train, y_train)

            # avalia o modelo no conjunto de treinamento
            y_train_pred = self.model.predict(X_train)
            train_accuracy = accuracy_score(y_train, y_train_pred)
            train_precision = precision_score(y_train, y_train_pred)
            train_recall = recall_score(y_train, y_train_pred)
            train_f1 = f1_score(y_train, y_train_pred)

            # avalia o modelo no conjunto de validação
            y_val_pred = self.model.predict(X_val)
            val_accuracy = accuracy_score(y_val, y_val_pred)
            val_precision = precision_score(y_val, y_val_pred)
            val_recall = recall_score(y_val, y_val_pred)
            val_f1 = f1_score(y_val, y_val_pred)

            # registra as métricas de avaliação
            mlflow.log_metric("train_accuracy", train_accuracy)
            mlflow.log_metric("train_precision", train_precision)
            mlflow.log_metric("train_recall", train_recall)
            mlflow.log_metric("train_f1", train_f1)
            mlflow.log_metric("val_accuracy", val_accuracy)
            mlflow.log_metric("val_precision", val_precision)
            mlflow.log_metric("val_recall", val_recall)
            mlflow.log_metric("val_f1", val_f1)

            # salva o modelo no MLflow
            mlflow.sklearn.log_model(self.model, "model")

    def predict(self, X_test):
        y_pred = self.model.predict(X_test)
        return y_pred.astype(int)


In [24]:
df2 = df.drop("Downgrade", axis=1)
df2.head()

,CPU Load (%),CPU Cores,Memory RAM Usage (%),Memory Swap Usage (%),Disk Usage (%),downgrade
0,86.81,2,86.34,79.05,54.58,1
1,82.48,4,99.86,31.85,84.87,1
2,70.12,4,94.80,11.67,93.15,1
3,19.17,4,20.22,55.62,14.25,1
4,41.22,3,88.39,3.89,82.50,1


In [60]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# separando as features e o target
X = df2.drop('downgrade', axis=1)
y = df2['downgrade']

# dividindo em conjunto de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# normalizando os dados
# scaler = StandardScaler()
# X_train_scaled = scaler.fit_transform(X_train)
# X_test_scaled = scaler.transform(X_test)

# criando o modelo de redes neurais
nn = NeuralNetwork()

# treinando o modelo
nn.fit(X_train, y_train, X_test, y_test)

# fazendo as previsões no conjunto de teste
y_pred = nn.predict(X_test)
y_pred
# avaliando o modelo
# accuracy = accuracy_score(y_test, y_pred)
# precision = precision_score(y_test, y_pred)
# recall = recall_score(y_test, y_pred)
# f1 = f1_score(y_test, y_pred)

# print('Accuracy: {:.2f}'.format(accuracy))
# print('Precision: {:.2f}'.format(precision))
# print('Recall: {:.2f}'.format(recall))
# print('F1-score: {:.2f}'.format(f1))


array([0, 1, 1, ..., 1, 1, 1])

In [62]:
X_new = [[80, 1, 60, 10, 50], [30, 2, 30, 5, 20]] # exemplo de novos dados
y_pred = nn.predict(X_new)
print(y_pred)

[1 0]


/home/kaique/.pyenv/versions/3.10.7/envs/mlops/lib/python3.10/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(


In [10]:
import mlflow
import mlflow.keras
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

class NeuralNetwork:
    def __init__(self, input_shape, learning_rate=0.001, threshold=0.5):
        self.input_shape = input_shape
        self.learning_rate = learning_rate
        self.threshold = threshold
        self.model = self._build_model()

    def _build_model(self):
        model = Sequential()
        model.add(Dense(16, input_shape=self.input_shape, activation='relu'))
        model.add(Dropout(0.2))
        model.add(Dense(8, activation='relu'))
        model.add(Dropout(0.2))
        model.add(Dense(1, activation='sigmoid'))
        optimizer = Adam(learning_rate=self.learning_rate)
        model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
        return model

    def train(self, x_train, y_train, epochs=10, batch_size=32, validation_data=None):
        with mlflow.start_run():
            mlflow.log_params({
                "input_shape": self.input_shape,
                "learning_rate": self.learning_rate,
                "threshold": self.threshold,
                "epochs": epochs,
                "batch_size": batch_size,
            })
            history = self.model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size,
                                     validation_data=validation_data)
            mlflow.keras.log_model(self.model, "model")
            y_pred = self.predict(x_train)
            self.log_metrics(y_train, y_pred, prefix="train")
            if validation_data is not None:
                x_val, y_val = validation_data
                y_val_pred = self.predict(x_val)
                self.log_metrics(y_val, y_val_pred, prefix="val")
            return history

    def predict(self, x):
        y_pred = self.model.predict(x)
        y_pred_class = np.where(y_pred > self.threshold, 1, 0)
        return y_pred_class

    def log_metrics(self, y_true, y_pred, prefix=""):
        metrics = {
            f"{prefix}_accuracy": accuracy_score(y_true, y_pred),
            f"{prefix}_precision": precision_score(y_true, y_pred),
            f"{prefix}_recall": recall_score(y_true, y_pred),
            f"{prefix}_f1": f1_score(y_true, y_pred),
        }
        mlflow.log_metrics(metrics)


2023-05-01 09:55:14.127243: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-01 09:55:14.133328: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-01 09:55:14.172187: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-01 09:55:14.173140: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-01 09:55:14.954450: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [40]:
df2.to_csv('dados.csv', index=False)

In [30]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# Carrega os dados
dados = np.loadtxt('dados.csv', delimiter=',')

# Separa as features e a classe
X = dados[:, :-1]
y = dados[:, -1]

In [31]:
# Divide o conjunto de dados em treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normaliza os dados
scaler = MinMaxScaler()
X_train_norm = scaler.fit_transform(X_train)
X_test_norm = scaler.transform(X_test)

In [33]:
# Cria e treina a rede neural
input_shape = (X_train_norm.shape[1],)
nn = NeuralNetwork(input_shape=input_shape)
nn.train(X_train_norm, y_train)

# Testa a rede neural
y_pred = nn.predict(X_test_norm)

Epoch 1/10
2500/2500 [==============================] - 10s 4ms/step - loss: 0.2142 - accuracy: 0.9087
Epoch 2/10
2500/2500 [==============================] - 11s 4ms/step - loss: 0.1214 - accuracy: 0.9520
Epoch 3/10
2500/2500 [==============================] - 9s 4ms/step - loss: 0.0896 - accuracy: 0.9659
Epoch 4/10
2500/2500 [==============================] - 10s 4ms/step - loss: 0.0695 - accuracy: 0.9732
Epoch 5/10
2500/2500 [==============================] - 8s 3ms/step - loss: 0.0566 - accuracy: 0.9780
Epoch 6/10
2500/2500 [==============================] - 10s 4ms/step - loss: 0.0452 - accuracy: 0.9833
Epoch 7/10
2500/2500 [==============================] - 10s 4ms/step - loss: 0.0372 - accuracy: 0.9865
Epoch 8/10
2500/2500 [==============================] - 6s 3ms/step - loss: 0.0324 - accuracy: 0.9882
Epoch 9/10
2500/2500 [==============================] - 5s 2ms/step - loss: 0.0289 - accuracy: 0.9898
Epoch 10/10
2500/2500 [==============================] - 7s 3ms/step - loss: 

2023/05/01 12:08:32 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.
2023-05-01 12:08:32.790606: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,16]
	 [[{{node inputs}}]]
2023-05-01 12:08:32.806943: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,8]
	 [[{{node inputs}}]]
2023-05-01 12:08:32.987571: I tensorflow/core/common_runtime/executor.cc:1197] [/d

INFO:tensorflow:Assets written to: /tmp/tmpfa89efch/model/data/model/assets


INFO:tensorflow:Assets written to: /tmp/tmpfa89efch/model/data/model/assets


625/625 [==============================] - 1s 1ms/step


In [38]:

unique, counts = np.unique(y_pred, return_counts=True)
dict(zip(unique, counts))

{0: 2525, 1: 17475}

In [41]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier

# carregar dados
data = pd.read_csv('dados.csv')

# separar dados de entrada (X) e saída (y)
X = data.drop('downgrade', axis=1)
y = data['downgrade']

# dividir dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# escalonar os dados de entrada
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# criar o modelo MLP
mlp = MLPClassifier(hidden_layer_sizes=(50, 30, 10), max_iter=1000)

# treinar o modelo usando os dados de treinamento
mlp.fit(X_train, y_train)

# avaliar o modelo usando os dados de teste
accuracy = mlp.score(X_test, y_test)
print(f'Acurácia: {accuracy:.2f}')


Acurácia: 1.00


In [42]:
mlp.predict(X_test)

array([1, 1, 1, ..., 1, 1, 1])